In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# ----------------------------------------------
# LOAD DATA
# ----------------------------------------------
nodes_df = pd.read_csv("updated_nodes_with_intersections.csv")
edges_df = pd.read_csv("Edges.csv")

# Example column names (edit these if your files differ)
# nodes_df columns: node_id, x, y
# edges_df columns: source, target, humanitarian_w, risk_w, distance_w

# ----------------------------------------------
# BUILD GRAPHS FOR EACH SCENARIO
# ----------------------------------------------
def build_graph(edges, weight_col):
    G = nx.DiGraph()   # directed is safer for route restrictions
    for _, row in edges.iterrows():
        G.add_edge(
            row["source"],
            row["target"],
            weight=row[weight_col]
        )
    return G


G_humanitarian = build_graph(edges_df, "humanitarian_w")
G_risk         = build_graph(edges_df, "risk_w")
G_distance     = build_graph(edges_df, "distance_w")   # may contain negative weights


# ----------------------------------------------
# SHORTEST PATH FUNCTION (chooses correct algorithm)
# ----------------------------------------------
def compute_shortest_path(G, source, target):
    # Check for negative weights
    has_negative = any(data['weight'] < 0 for _,_,data in G.edges(data=True))

    if has_negative:
        print("Negative weights detected → using Bellman–Ford")
        path = nx.bellman_ford_path(G, source, target, weight='weight')
        length = nx.bellman_ford_path_length(G, source, target, weight='weight')
    else:
        print("No negative weights → using Dijkstra")
        path = nx.dijkstra_path(G, source, target, weight='weight')
        length = nx.dijkstra_path_length(G, source, target, weight='weight')

    return path, length


# ----------------------------------------------
# VISUALIZATION HELPERS
# ----------------------------------------------
def draw_graph(G, path):
    pos = {row["node_id"]: (row["x"], row["y"]) for _, row in nodes_df.iterrows()}

    plt.figure(figsize=(10, 8))

    # Draw all nodes & edges
    nx.draw(G, pos, node_size=30, edge_color='gray', alpha=0.6)

    # Highlight shortest path
    path_edges = list(zip(path, path[1:]))
    nx.draw_networkx_nodes(G, pos, nodelist=path, node_color='red', node_size=80)
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=2)

    plt.title("Shortest Path")
    plt.show()


# ----------------------------------------------
# EXAMPLE USAGE
# ----------------------------------------------
source_node = 1
target_node = 50  # change these

# Scenario 1: maximize humanitarian aid access
path_h, dist_h = compute_shortest_path(G_humanitarian, source_node, target_node)
print("Humanitarian path:", path_h, "cost:", dist_h)
draw_graph(G_humanitarian, path_h)

# Scenario 2: minimize risk
path_r, dist_r = compute_shortest_path(G_risk, source_node, target_node)
print("Risk path:", path_r, "cost:", dist_r)
draw_graph(G_risk, path_r)

# Scenario 3: distance-based (may be negative → Bellman-Ford)
path_d, dist_d = compute_shortest_path(G_distance, source_node, target_node)
print("Distance path:", path_d, "cost:", dist_d)
draw_graph(G_distance, path_d)


KeyError: 'source'

In [11]:
import pandas as pd
import numpy as np
import networkx as nx

# -----------------------------------------------------
# 1. LOAD NODES
# -----------------------------------------------------
try:
    nodes_df = pd.read_csv("updated_nodes_with_intersections.csv")
    print("Nodes loaded:", nodes_df.shape)
except FileNotFoundError:
    print("Error: Nodes file not found. Please check filename.")
    exit()

# -----------------------------------------------------
# 2. LOAD EXCEL (SIDE-BY-SIDE TABLES)
# -----------------------------------------------------
excel_file = "your_edges_file.xlsx"

# Read without header first to accurately find the separator
raw = pd.read_excel(excel_file, header=None)
print("Raw Excel shape:", raw.shape)

# -----------------------------------------------------
# 3. ROBUST SPLIT (FIND EMPTY SEPARATOR COLUMN OR ASSUME WIDTH)
# -----------------------------------------------------

# Find the first column that is completely empty (all NaNs)
empty_cols = raw.columns[raw.isna().all()].tolist()

# Define the expected number of columns for the edges table
EXPECTED_EDGE_COLUMNS = 3 

if not empty_cols:
    # Scenario 1: No empty column found (tables are side-by-side)
    # Assume the edges table is the first 3 columns, as expected.
    split_index = EXPECTED_EDGE_COLUMNS 
    print(f"Warning: No empty separator column found. Assuming split after column {split_index}.")
elif empty_cols[0] == 0:
    # Scenario 2: The very first column is empty (highly unlikely for data)
    # This might indicate a problem in the raw file read or a completely blank sheet.
    raise ValueError("The first column of the Excel sheet appears empty. Check the sheet data.")
else:
    # Scenario 3: Empty separator found at index > 0 (Success)
    split_index = empty_cols[0]
    # Check if the found split index is reasonable (at least 3 columns for edges)
    if split_index < EXPECTED_EDGE_COLUMNS:
        print(f"Warning: Found separator at index {split_index}, but expected at least 3 edge columns. Assuming split is {EXPECTED_EDGE_COLUMNS}.")
        split_index = EXPECTED_EDGE_COLUMNS

print(f"Final determined split index for Edges/Scenarios: {split_index}")

# -----------------------------------------------------
# 4. ROBUST SPLIT AND HEADER ASSIGNMENT (NO CHANGES NEEDED HERE, IT'S CORRECT NOW)
# -----------------------------------------------------

# -----------------------------------------------------
# 5. CLEAN DATA TYPES AND RENAME EDGE COLUMNS (UPDATED)
# -----------------------------------------------------
# Expecting: Source | Target | Weight
# The column assignment relies on the headers being correct from the Excel file
# We rename them for use in the networkx part

if edges_df.shape[1] < 3:
    raise ValueError(f"Edges table has only {edges_df.shape[1]} columns. Expected 3 (Source, Target, Weight).")

edges_df.columns = ["Source", "Target", "Weight"]

edges_df["Source"] = edges_df["Source"].astype(str).str.strip()
edges_df["Target"] = edges_df["Target"].astype(str).str.strip()
edges_df["Weight"] = pd.to_numeric(edges_df["Weight"], errors="coerce").fillna(0)

# -----------------------------------------------------
# 4. CREATE DATAFRAMES
# -----------------------------------------------------
# LEFT TABLE (Edges): Columns 0 to split_index
edges_raw = raw.iloc[:, :split_index].copy()

# RIGHT TABLE (Scenarios): Columns after split_index
scenarios_raw = raw.iloc[:, split_index+1:].copy()

# --- FIX HEADERS FOR LEFT TABLE ---
edges_df = edges_raw.iloc[1:].copy() # Data starts at row 1
edges_df.columns = edges_raw.iloc[0] # Header is row 0

# --- FIX HEADERS FOR RIGHT TABLE ---
scenarios_df = scenarios_raw.iloc[1:].copy() # Data starts at row 1
scenarios_df.columns = scenarios_raw.iloc[0] # Header is row 0

# Drop empty rows
edges_df.dropna(how='all', inplace=True)
scenarios_df.dropna(how='all', inplace=True)

print(f"Edges Table: {edges_df.shape}")
print(f"Scenarios Table: {scenarios_df.shape}")

# -----------------------------------------------------
# 5. CLEAN DATA TYPES
# -----------------------------------------------------
# Standardize Edge Columns
# We expect the first 3 columns to be Source, Target, Weight
edges_df.columns = ["Source", "Target", "Weight"] 

edges_df["Source"] = edges_df["Source"].astype(str).str.strip()
edges_df["Target"] = edges_df["Target"].astype(str).str.strip()
edges_df["Weight"] = pd.to_numeric(edges_df["Weight"], errors="coerce").fillna(0)

# Standardize Scenario Columns
# Assuming Right Table is: [Scenario Name, Coefficient]
if scenarios_df.shape[1] >= 2:
    scenarios_df.columns = ["Scenario", "Coefficient"]
    scenarios_df["Coefficient"] = pd.to_numeric(scenarios_df["Coefficient"], errors="coerce").fillna(0)
else:
    print("Warning: Scenarios table does not have 2 columns. Check Excel format.")

# -----------------------------------------------------
# 6. BUILD GRAPH
# -----------------------------------------------------
G = nx.DiGraph()

# Add nodes with error checking for missing columns in nodes CSV
required_cols = ["Name", "X", "Y", "Risk", "HumanitarianAid", "Type"]
for col in required_cols:
    if col not in nodes_df.columns:
        print(f"Error: Node CSV missing column '{col}'")
        exit()

for _, row in nodes_df.iterrows():
    G.add_node(str(row["Name"]).strip(), 
               x=row["X"], 
               y=row["Y"], 
               risk=pd.to_numeric(row["Risk"], errors='coerce'),
               humanitarian=pd.to_numeric(row["HumanitarianAid"], errors='coerce'),
               type=row["Type"])

# Add edges
for _, row in edges_df.iterrows():
    G.add_edge(row["Source"], row["Target"], weight=row["Weight"])

print(f"Graph built: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# -----------------------------------------------------
# 7. RUN SCENARIOS
# -----------------------------------------------------
output_rows = []

# Iterate through the Scenarios DataFrame (Rows, not columns)
for _, row in scenarios_df.iterrows():
    scen_name = row["Scenario"]
    coeff = row["Coefficient"]

    total_risk_score = 0
    
    for n in G.nodes:
        # Safety check: ensure node has attributes (handle NaNs)
        node_risk = G.nodes[n].get("risk", 0) or 0
        node_hum = G.nodes[n].get("humanitarian", 0) or 0
        
        # Calculation logic
        combined_val = (float(node_risk) * float(coeff)) + float(node_hum)
        total_risk_score += combined_val

    output_rows.append({
        "Scenario": scen_name,
        "Coefficient": coeff,
        "TotalMetric": total_risk_score
    })

results_df = pd.DataFrame(output_rows)

# -----------------------------------------------------
# 8. SAVE OUTPUT
# -----------------------------------------------------
try:
    results_df.to_csv("scenario_results.csv", index=False)
    print("\nSuccess! 'scenario_results.csv' saved.")
    print(results_df.head())
except Exception as e:
    print(f"Error saving file: {e}")

Nodes loaded: (40, 6)
Raw Excel shape: (0, 0)
Final determined split index for Edges/Scenarios: 3


ValueError: Edges table has only 0 columns. Expected 3 (Source, Target, Weight).

In [ ]:
import pandas as pd
import networkx as nx

# ---------------------------------------------------------
# LOAD DATA
# ---------------------------------------------------------
df = pd.read_csv("Edges.csv")

# Clean accidental unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# ---------------------------------------------------------
# BUILD DIRECTED GRAPH
# ---------------------------------------------------------
G = nx.DiGraph()

# Add edges with all three weight types
for _, row in df.iterrows():
    G.add_edge(
        row['From'],
        row['To'],
        risk_weight=float(row['danger weights']),
        aid_weight=float(row['humanitarian weights']),
        dist_weight=float(row['distance weights'])
    )

# ---------------------------------------------------------
# ALGORITHM SELECTOR
# ---------------------------------------------------------
def use_dijkstra(weight_type):
    return nx.dijkstra_path(G, source_node, target_node, weight=weight_type)

def use_bellman(weight_type):
    return nx.bellman_ford_path(G, source_node, target_node, weight_type)

def compute_path(source_node, target_node, mode):
    """
    mode = "risk" → uses Dijkstra
    mode = "aid"  → humanitarian priority → negative weights → Bellman–Ford
    mode = "distance" → shortest distance → Dijkstra
    """

    if mode == "risk":
        weight_key = "risk_weight"
        algorithm = "dijkstra"

    elif mode == "aid":
        weight_key = "aid_weight"
        algorithm = "bellman"

    elif mode == "distance":
        weight_key = "dist_weight"
        algorithm = "dijkstra"

    else:
        raise ValueError("Invalid mode. Choose: risk, aid, distance")

    print(f"\n➡ Mode selected: {mode.upper()}")
    print(f"   Using algorithm: {algorithm}")

    try:
        if algorithm == "dijkstra":
            path = nx.dijkstra_path(G, source_node, target_node, weight=weight_key)
            cost = nx.dijkstra_path_length(G, source_node, target_node, weight=weight_key)
        else:
            path = nx.bellman_ford_path(G, source_node, target_node, weight_key)
            cost = nx.bellman_ford_path_length(G, source_node, target_node, weight_key)

        return path, cost

    except nx.NetworkXNoPath:
        return None, None

# ---------------------------------------------------------
# EXAMPLE: USER SELECTS THEIR CONDITION
# ---------------------------------------------------------
source_node = "Beit Hanoun"
target_node = "Rafah int"

user_condition = input("Choose your situation (risk / aid / distance): ").strip()

path, total_cost = compute_path(source_node, target_node, user_condition)

if path:
    print("\nOptimal Path Found:")
    print(" → ".join(path))
    print(f"Total cost: {total_cost}")
else:
    print("❌ No valid path found for this mode.")

In [4]:
import pandas as pd
import networkx as nx

# ===========================
# 1. LOAD NODES
# ===========================
nodes_df = pd.read_csv("updated_nodes_with_intersections.csv")

# Clean column names
nodes_df.columns = nodes_df.columns.str.strip()

# Normalize type and name
nodes_df["Type"] = nodes_df["Type"].str.lower().str.strip()
nodes_df["Name"] = nodes_df["Name"].astype(str).str.strip()

# ===========================
# 2. LOAD EDGES
# ===========================
edges_df = pd.read_csv("Edges.csv")
edges_df = edges_df.loc[:, ~edges_df.columns.str.contains('^Unnamed')]

# Ensure numeric weights
weight_cols = ["danger weights", "humanitarian weights", "distance weights"]
for col in weight_cols:
    edges_df[col] = pd.to_numeric(edges_df[col], errors='coerce').fillna(0)

# Clean node names
edges_df['From'] = edges_df['From'].astype(str).str.strip()
edges_df['To'] = edges_df['To'].astype(str).str.strip()

# Shift negative weights to positive to avoid Bellman-Ford errors
for col in weight_cols:
    min_val = edges_df[col].min()
    if min_val < 0:
        edges_df[col] += abs(min_val) + 0.01  # small epsilon to avoid zero

# ===========================
# 3. BUILD GRAPH
# ===========================
G = nx.DiGraph()

# Add edges with all weights and make undirected
for _, row in edges_df.iterrows():
    for a, b in [(row['From'], row['To']), (row['To'], row['From'])]:
        G.add_edge(a, b,
                   risk=row['danger weights'],
                   aid=row['humanitarian weights'],
                   dist=row['distance weights'])

# ===========================
# 4. DETECT CIVILIAN & DESTINATION NODES
# ===========================
civilians = nodes_df[nodes_df["Type"] == "civilians"]["Name"].tolist()
destinations = nodes_df[nodes_df["Type"] == "destination"]["Name"].tolist()

print("Civilian nodes:", civilians)
print("Destination nodes:", destinations)

# ===========================
# 5. PATH COMPUTATION FUNCTION
# ===========================
def compute_path(src, dst, weight_type):
    """
    Computes shortest path from src to dst using Dijkstra.
    Safely handles nodes not in graph.
    """
    if src not in G or dst not in G:
        return None, None, None

    try:
        path = nx.dijkstra_path(G, src, dst, weight=weight_type)
        cost = nx.dijkstra_path_length(G, src, dst, weight=weight_type)
        algo = "dijkstra"
        return path, cost, algo
    except nx.NetworkXNoPath:
        return None, None, None

# ===========================
# 6. COMPUTE ALL PATHS
# ===========================
scenarios = {
    "risk": "risk",
    "aid": "aid",
    "distance": "dist"
}

all_results = []

for scenario, weight_col in scenarios.items():
    print(f"\n>>> Running scenario: {scenario.upper()}")
    results = []

    for src in civilians:
        for dst in destinations:
            path, cost, algo = compute_path(src, dst, weight_col)
            results.append({
                "scenario": scenario,
                "source": src,
                "target": dst,
                "path": path,
                "cost": cost,
                "algorithm": algo,
                "note": "ok" if path else "no path"
            })

    df_results = pd.DataFrame(results)
    df_results.to_csv(f"paths_results_{scenario}.csv", index=False)
    all_results.append(df_results)

# ===========================
# 7. COMBINE ALL SCENARIOS
# ===========================
final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv("paths_results_all_scenarios.csv", index=False)

print("\n✅ All scenario CSVs generated successfully.")


Civilian nodes: ['Beit Hanoun', 'Beit Lahia', 'Jabalia', 'Atatra', 'Al-Shati', 'Rimal', 'Tuffah neighborhood park', "Shuja'iyya", 'Zeitoun', 'Sabra', 'Sheikh Radwan', 'Nuseirat', 'Bureij', 'Maghazi', 'Az Zawayda', 'Deir al Balah', 'Al Musaddar', 'Abasan al-Kabira', 'Abasan al-Saghira', 'Bani Suheila', "Khuza'a", 'Al Qarara', 'Al Fukhkhari', 'Rafah Camp', 'Tal as Sultan', 'مستشفي الخير', 'مدرسة الفاخورة']
Destination nodes: ['مستشفى شهداء الاقصى', 'Gaza field hospital of the Jordanian-', 'مستشفى الشفا - مبنى الحروق', 'UNRWA vocational college', 'UNRWA-GFO', 'Al Quds Open University - Gaza Branch', 'جامعة الازهر كلية الحقوق المغراقة', 'Islamic University - khanyonuis']

>>> Running scenario: RISK

>>> Running scenario: AID

>>> Running scenario: DISTANCE

✅ All scenario CSVs generated successfully.


In [5]:
import pandas as pd
import networkx as nx

# -------------------------------------------------------
# 1. LOAD DATA
# -------------------------------------------------------
df = pd.read_csv("Edges.csv")

# small cleanup
df = df.dropna(subset=["From", "To"])

# -------------------------------------------------------
# 2. BUILD GRAPH
# -------------------------------------------------------
G = nx.DiGraph()

for _, row in df.iterrows():
    G.add_edge(
        row["From"],
        row["To"],
        risk_weight=row["danger weights"],
        aid_weight=row["humanitarian weights"],
        distance_weight=row["distance weights"]
    )

# -------------------------------------------------------
# 3. USER INTERFACE
# -------------------------------------------------------
print("\n========================")
print("PALESTINE SAFE ROUTE FINDER")
print("========================\n")

print("Select your starting location:")
nodes = sorted(list(G.nodes()))
for i, n in enumerate(nodes):
    print(f"{i+1}. {n}")

start_choice = int(input("\nEnter number: ")) - 1
start_node = nodes[start_choice]

print("\nChoose your priority:")
print("1. Avoid RISK (safest path)")
print("2. Seek HUMANITARIAN AID (injured case)")
print("3. Prioritise DISTANCE (fastest path)")

case_choice = int(input("\nEnter number: "))

if case_choice == 1:
    weight_type = "risk_weight"
    print("\n➡ Priority: Safety / Avoid risk")
elif case_choice == 2:
    weight_type = "aid_weight"
    print("\n➡ Priority: Humanitarian Aid")
elif case_choice == 3:
    weight_type = "distance_weight"
    print("\n➡ Priority: Distance")
else:
    raise ValueError("Invalid choice!")

# -------------------------------------------------------
# 4. CHOOSE ALGORITHM BASED ON WEIGHTS
# -------------------------------------------------------
# extract weights from graph
weights = [data[weight_type] for u, v, data in G.edges(data=True)]

use_bellmanford = any(w < 0 for w in weights)

print("\nSelected algorithm:")
if use_bellmanford:
    print("➡ Bellman–Ford (negative weights detected)")
else:
    print("➡ Dijkstra (all weights positive)")

# -------------------------------------------------------
# 5. CALCULATE SHORTEST PATHS
# -------------------------------------------------------
results = {}
for target in nodes:
    if target == start_node:
        continue
    try:
        if use_bellmanford:
            length, path = nx.single_source_bellman_ford(G, start_node, target, weight=weight_type)
        else:
            length = nx.dijkstra_path_length(G, start_node, target, weight=weight_type)
            path = nx.dijkstra_path(G, start_node, target, weight=weight_type)

        results[target] = (length, path)
    except:
        pass

# -------------------------------------------------------
# 6. DISPLAY BEST DESTINATION
# -------------------------------------------------------
print("\n========================")
print("Recommended Safe Destination")
print("========================")

best_dest = min(results, key=lambda x: results[x][0])
best_cost, best_path = results[best_dest]

print(f"\nBest Destination: {best_dest}")
print(f"Total Cost: {best_cost:.3f}")
print(f"Path: {' → '.join(best_path)}")
print("\n========================\n")


PALESTINE SAFE ROUTE FINDER

Select your starting location:
1. Abasan al-Kabira
2. Abasan al-Saghira
3. Al Fukhkhari
4. Al Musaddar
5. Al Qarara
6. Al Quds Open University - Gaza Branch
7. Al-Shati
8. Atatra
9. Az Zawayda
10. Bani Suheila
11. Beit Hanoun
12. Beit Lahia
13. Bureij
14. Deir al Balah
15. Gaza field hospital of the Jordanian-
16. Islamic University - khanyonuis
17. Jabalia
18. Khuza'a
19. Maghazi
20. Nuseirat
21. Rafah Camp
22. Rafah int
23. Rimal
24. Sabra
25. Sheikh Radwan
26. Shuja'iyya
27. Tal as Sultan
28. Tuffah neighborhood park
29. UNRWA vocational college
30. UNRWA-GFO
31. Zeitoun
32. gaza city int
33. khan younes int
34. middle camps int
35. north gaza int
36. جامعة الازهر كلية الحقوق المغراقة
37. مدرسة الفاخورة
38. مستشفى الشفا - مبنى الحروق
39. مستشفى شهداء الاقصى
40. مستشفي الخير


ValueError: invalid literal for int() with base 10: ''